In [3]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print("CUDA is available. PyTorch can use the GPU.")
    print("Number of available GPUs:", torch.cuda.device_count())
    print("Current GPU device:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. PyTorch will use the CPU.")

CUDA is available. PyTorch can use the GPU.
Number of available GPUs: 1
Current GPU device: 0
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [4]:
import torch

# Before starting the training loop
torch.cuda.empty_cache()

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import transforms
import torch.optim as optim
import os
import cv2
from glob import glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix

In [6]:
dataset_path = "C:/Users/91623/Desktop/Brain_tumor_research/cleaned"
train_path = "C:/Users/91623/Desktop/Brain_tumor_research/cleaned/Training"
test_path = "C:/Users/91623/Desktop/Brain_tumor_research/cleaned/Testing"
image_size = (256,256)
num_classes = 5
train_paths = glob(f"{train_path}/*/*.jpg")
test_paths = glob(f"{test_path}/*/*.jpg")

In [7]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
# class dataset(Dataset):
#     def __init__(self,paths,transform=None):
#         self.paths = paths
#         self.transform = transform
#         self.labels = [self.get_label(path) for path in self.paths]
#         self.labels_to_index = {label:index for index,label in enumerate(list(set(self.labels)))}
    
#     def __getitem__(self,index):
#         img_path = self.paths[index]
#         img = Image.open(img_path).convert('RGB') 
#         label = self.labels[index]
#         label_index = self.labels_to_index[label]
        
#         if self.transform:
#             img = self.transform(img)
        
#         return img, label_index,label
    
    
#     def __len__(self):
#         return len(self.paths)
    
    
#     def get_label(self, path):
#         return os.path.basename(os.path.dirname(path))


In [9]:
# train_dataset = dataset(train_paths, transform=train_transform)
# test_dataset = dataset(test_paths, transform=test_transform)

In [10]:
# val_dataset, test_dataset = train_test_split(test_dataset,test_size=0.5, random_state=ord("H"))

In [11]:
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [12]:
# import torch
# import torch.nn as nn
# from torchvision import models
# from torch.optim.lr_scheduler import ReduceLROnPlateau

# # Define the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# # Load the ResNet152V2 model with pre-trained weights
# resnet = models.resnet152(pretrained=True)

# # Freeze the existing weights
# for param in resnet.parameters():
#     param.requires_grad = False

# # Modify the final layers to match your classification task
# # Replace the fully connected layer with a new one
# class CustomResNet152(nn.Module):
#     def __init__(self):
#         super(CustomResNet152, self).__init__()
#         self.resnet = resnet
#         self.resnet.fc = nn.Sequential(
#             nn.Linear(resnet.fc.in_features, 256),
#             nn.ReLU(),
#             nn.Linear(256, 4),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         return self.resnet(x)

# # Instantiate the model
# model = CustomResNet152().to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, min_lr=0.001)

# # Print the model architecture
# print(model)


In [13]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.datasets import ImageFolder

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Load the ResNet152 model with pre-trained weights
resnet = models.resnet152(pretrained=True)

# Freeze the existing weights
for param in resnet.parameters():
    param.requires_grad = False

# Modify the final layers to match your classification task
class CustomResNet152(nn.Module):
    def __init__(self):
        super(CustomResNet152, self).__init__()
        self.resnet = resnet
        self.resnet.fc = nn.Sequential(
            nn.Linear(resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 4),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.resnet(x)

# Instantiate the model
model = CustomResNet152().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, min_lr=0.001)

# Print the model architecture
print(model)

# Prepare the data (example using ImageFolder, adjust paths as needed)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = ImageFolder(root=train_path, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

val_dataset = ImageFolder(root=test_path, transform=train_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


c:\Users\91623\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\91623\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomResNet152(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [14]:
def train_epoch():
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_predictions = 0
    
    for batch_number, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        
        loss.backward()
        optimizer.step()
        
        if (batch_number and batch_number % 150 == 0):
            c_loss = total_loss / (batch_number + 1)
            accuracy = total_correct / total_predictions
            print(f"Batch {batch_number}/{len(train_loader)}, Loss: {c_loss:0.4f}, Accuracy: {accuracy * 100:.2f}%")
    
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_correct / total_predictions
    
    return avg_loss, avg_accuracy

In [15]:
def validate_epoch():
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_predictions = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            total_correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(val_loader)
    avg_accuracy = total_correct / total_predictions
    
    return avg_loss, avg_accuracy

num_epochs = 15

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch()
    val_loss, val_accuracy = validate_epoch()
    
    scheduler.step(val_loss)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy*100:.2f}%')
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy*100:.2f}%')


Batch 150/179, Loss: 0.9846, Accuracy: 77.50%
Epoch 1/15, Train Loss: 0.9723, Train Accuracy: 78.61%
Epoch 1/15, Validation Loss: 0.9310, Validation Accuracy: 81.77%
Batch 150/179, Loss: 0.8916, Accuracy: 85.68%
Epoch 2/15, Train Loss: 0.8920, Train Accuracy: 85.56%
Epoch 2/15, Validation Loss: 0.8993, Validation Accuracy: 84.29%
Batch 150/179, Loss: 0.8716, Accuracy: 87.19%
Epoch 3/15, Train Loss: 0.8729, Train Accuracy: 87.08%
Epoch 3/15, Validation Loss: 0.8963, Validation Accuracy: 85.28%
Batch 150/179, Loss: 0.8679, Accuracy: 87.60%
Epoch 4/15, Train Loss: 0.8673, Train Accuracy: 87.69%
Epoch 4/15, Validation Loss: 0.9026, Validation Accuracy: 83.98%
Batch 150/179, Loss: 0.8523, Accuracy: 89.07%
Epoch 5/15, Train Loss: 0.8595, Train Accuracy: 88.41%
Epoch 5/15, Validation Loss: 0.9297, Validation Accuracy: 81.08%
Batch 150/179, Loss: 0.8574, Accuracy: 88.49%
Epoch 6/15, Train Loss: 0.8586, Train Accuracy: 88.36%
Epoch 6/15, Validation Loss: 0.8877, Validation Accuracy: 85.74%
Batc

In [16]:
# def eval_model():
#     model.eval()
#     total_loss = 0.0 
#     total_correct = 0
#     total_predictions = 0
    
#     with torch.no_grad():
#         for batch_number, (images, labels_indices, labels) in enumerate(val_loader):
#             images, labels_indices = images.to(device), labels_indices.to(device)
            
#             outputs = model(images)
#             loss = criterion(outputs, labels_indices)
#             total_loss += loss.item()
            
#             # Calculate accuracy
#             _, predicted = torch.max(outputs, 1)
#             total_predictions += labels_indices.size(0)
#             total_correct += (predicted == labels_indices).sum().item()
            
#     avg_loss = total_loss / len(val_loader)
#     avg_accuracy = total_correct / total_predictions
    
#     return avg_loss, avg_accuracy

In [17]:
# train_losses = []
# val_losses = []
# train_accuracies = []
# val_accuracies = []
# best_score = float("inf")
# n_epochs = 40
# tolerance = 0

# for epoch in range(n_epochs):
#     print(f"Epoch {epoch},   ", end="")
#     avg_train_loss, train_accuracy = train_epoch()
#     avg_eval_loss, eval_accuracy = eval_model()
    
#     if avg_eval_loss < best_score:
#         best_score = avg_eval_loss
#         torch.save(model.state_dict(), f"model_{epoch+1}")
#         tolerance = 0
#     else:
#         tolerance += 1
#         if tolerance >= 8:
#             break
            
#     train_losses.append(avg_train_loss)
#     val_losses.append(avg_eval_loss)
#     train_accuracies.append(train_accuracy)
#     val_accuracies.append(eval_accuracy)
    
#     print(f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%")
#     print(f"Eval Loss: {avg_eval_loss:.4f}, Eval Accuracy: {eval_accuracy * 100:.2f}%")